In [ ]:
import os
import os.path as op
import re
import zipfile
import glob
from itertools import chain, islice
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%ls data

In [ ]:
def _csv_filename(zip_filename):
    return op.splitext(op.basename(zip_filename))[0]

In [ ]:
def _iter_lines(zip_filename):
    print("Processing file {file}...".format(file=zip_filename))
    csv_filename = _csv_filename(zip_filename)
    with zipfile.ZipFile(zip_filename) as z:
        with z.open(csv_filename) as f:
            for line in f:
                yield line

In [ ]:
def _iter_all_lines(files, step=None, stop=None):
    return chain(*[islice(_iter_lines(f), min(1, i), stop, step)
                   for i, f in enumerate(files)])

In [ ]:
def _extract_number(filename):
    r = re.search(r'([\d]+)', filename)
    if r:
        return int(r.group(1))

In [ ]:
def _zip_filenames(name):
    return sorted(glob.glob('data/trip_{name}_*.zip'.format(name=name)), 
                  key=_extract_number)

In [ ]:
def _make_extract(step=None, stop=None):
    for name in ('fare', 'data'):
        filename = 'data/trip_{name}_subset.csv'.format(name=name)
        files = _zip_filenames(name)
        with open(filename, 'wb') as f:
            for line in _iter_all_lines(files, step=step, stop=stop):
                f.write(line)

In [ ]:
# Only keep one out of 'step' lines.
step = 200
# In every file, stop after 'stop' lines (None=until the end).
stop = 1000

In [ ]:
_make_extract(step=step, stop=None)